# 5.2 参数管理
我们下面关注操作参数的具体细节：
- 访问参数，用于调试，诊断，可视化；
- 参数初始化；
- 在不同模型组件之间共享参数


In [5]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), 
                    nn.ReLU(), 
                    nn.Linear(8, 1))

X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0356],
        [0.0288]], grad_fn=<AddmmBackward0>)

In [7]:
## 5.2.1 参数访问
print(net[2].state_dict())
# 我们在这里可以通过索引访问模型的任意层，就像是列表一样

OrderedDict([('weight', tensor([[-0.0571,  0.3135, -0.3087, -0.3134,  0.1762, -0.2426, -0.1297, -0.2376]])), ('bias', tensor([0.0630]))])


In [8]:
# 注意每一个参数都表示[参数类]的一个实例，要想操作需要访问底层的数值
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0630], requires_grad=True)
tensor([0.0630])


> 注：参数本身是复合的对象，包含了值，梯度，额外信息等等，有时候我们需要**显式**参数值

In [9]:
net[2].weight.grad == None

True

In [10]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
#  *[...]这是python语法：列表推导式
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
# 由此可见，我们访问网络参数可借用名称
net.state_dict()['2.bias'].data

tensor([0.0630])

### 从嵌套块收集参数

In [20]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                         nn.Linear(8,4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 进行嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[-0.1934],
        [-0.1934]], grad_fn=<AddmmBackward0>)

In [21]:
print(rgnet)
# 这里可以比较显然的看出，模型内部是有类似于递归嵌套的关系

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [26]:
print(rgnet[0])
print("-----------------")
print(rgnet[0][1])
print("-----------------")
print(rgnet[0][1][0])
rgnet[0][1][0].weight.data
# in 4 out 8 -> weight:4*8 bias:8

Sequential(
  (block 0): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
  (block 1): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
  (block 2): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
  (block 3): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
)
-----------------
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
)
-----------------
Linear(in_features=4, out_features=8, bias=True)


tensor([[-0.3776,  0.1784,  0.3345,  0.1383],
        [ 0.0346, -0.0722,  0.0191,  0.4177],
        [-0.2916, -0.0995, -0.4167, -0.1662],
        [ 0.3867, -0.4857, -0.3675,  0.4406],
        [ 0.1097,  0.2844,  0.1520, -0.1645],
        [ 0.0091,  0.0481, -0.4650,  0.4348],
        [-0.3623, -0.0237,  0.3339, -0.3785],
        [-0.0992,  0.2650, -0.1357,  0.0554]])

## 5.2.2 参数初始化

In [28]:
# 内置初始化
def init_normal(m):
    """高斯随机变量初始化"""
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0026,  0.0084, -0.0089,  0.0118]), tensor(0.))

In [30]:
def init_constant(m):
    """初始化为常数"""
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [32]:
# 我们还可以对不同的块应用不同的初始化方法
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0746, -0.2409,  0.5661,  0.6185])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [37]:
# 自定义初始化
# 当我们希望使用特定的分布进行初始化时
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                            for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -6.5335, -0.0000],
        [ 0.0000,  9.8022,  8.0159, -0.0000]], grad_fn=<SliceBackward0>)

In [38]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -5.5335,  1.0000])

## 5.2.3 参数绑定
有时，我们希望在多个层间**共享参数**
比如说：我们可以定义一个稠密层，然后用他的参数来设置另一个层的参数

In [40]:
# 我们需要【给共享层一个名称】，以便可以引用他的参数？？
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是[同一个对象]，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 练习
1. 为什么共享参数是个好主意：
   例如卷积神经网络通过使用卷积核共享参数；循环神经网络，transformer模型中的自注意力机制